In [2]:
## Fundamentals of Libraries ####

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from pydantic import BaseModel, Field

## Google Libraries ####

import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas_gbq
import pandasql
from pandas_gbq import read_gbq
from google.cloud import secretmanager
import json
import os

## LANGCHAIN LIBS ####

import openai
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool  # Example tool for search capabilities
from langchain.tools import DuckDuckGoSearchRun 
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.tools import DuckDuckGoSearchResults
from crewai_tools import Tool



In [3]:
##pip install crewai crewai-tools

In [4]:
##Get Secret Key - open-api-key From GCP Secret Manager

def get_secret(secret_name, project_id, version_id='1'):
    client = secretmanager.SecretManagerServiceClient()
    secret_path = f"projects/{project_id}/secrets/{secret_name}/versions/{version_id}"
    response = client.access_secret_version(name=secret_path)
    return response.payload.data.decode('UTF-8')

In [5]:
project_id = "psychic-root-424207-s9"
openai_api_key = get_secret("openai-api-key", project_id)
secret_id_service_account = "myfirstproject02_secretman"
dataset = "ga4_obfuscated_sample_ecommerce"

In [6]:
secret_payload = get_secret(secret_id_service_account, project_id)
gcp_credentials = json.loads(secret_payload)
credentials = service_account.Credentials.from_service_account_info(gcp_credentials)

In [7]:
client = bigquery.Client(credentials=credentials, project='bigquery-public-data')

In [8]:
dataset_ref = client.dataset('ga4_obfuscated_sample_ecommerce', project='bigquery-public-data')
tables = client.list_tables(dataset_ref)

# Get Schema
for table in tables:
    print(table.table_id)

events_20201101
events_20201102
events_20201103
events_20201104
events_20201105
events_20201106
events_20201107
events_20201108
events_20201109
events_20201110
events_20201111
events_20201112
events_20201113
events_20201114
events_20201115
events_20201116
events_20201117
events_20201118
events_20201119
events_20201120
events_20201121
events_20201122
events_20201123
events_20201124
events_20201125
events_20201126
events_20201127
events_20201128
events_20201129
events_20201130
events_20201201
events_20201202
events_20201203
events_20201204
events_20201205
events_20201206
events_20201207
events_20201208
events_20201209
events_20201210
events_20201211
events_20201212
events_20201213
events_20201214
events_20201215
events_20201216
events_20201217
events_20201218
events_20201219
events_20201220
events_20201221
events_20201222
events_20201223
events_20201224
events_20201225
events_20201226
events_20201227
events_20201228
events_20201229
events_20201230
events_20201231
events_20210101
events_2

In [9]:
# Örnek olarak bir tablo seçelim
table_id = 'events_20210131'

# Tablo referansı
table_ref = dataset_ref.table(table_id)

# Tabloyu alın
table = client.get_table(table_ref)

# Şemayı alın
schema = table.schema

# Şema bilgilerini yazdırın
for field in schema:
    print(f"{field.name} ({field.field_type})")


event_date (STRING)
event_timestamp (INTEGER)
event_name (STRING)
event_params (RECORD)
event_previous_timestamp (INTEGER)
event_value_in_usd (FLOAT)
event_bundle_sequence_id (INTEGER)
event_server_timestamp_offset (INTEGER)
user_id (STRING)
user_pseudo_id (STRING)
privacy_info (RECORD)
user_properties (RECORD)
user_first_touch_timestamp (INTEGER)
user_ltv (RECORD)
device (RECORD)
geo (RECORD)
app_info (RECORD)
traffic_source (RECORD)
stream_id (INTEGER)
platform (STRING)
event_dimensions (RECORD)
ecommerce (RECORD)
items (RECORD)


In [10]:
sql = """

SELECT distinct event_name 
FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` 
"""
event_names  = read_gbq(sql, project_id=project_id, credentials=credentials)
event_names.head(15) 

Downloading: 100%|██████████|


,event_name
0,page_view
1,scroll
2,select_promotion
3,purchase
4,view_promotion
5,select_item
6,view_item
7,session_start
8,user_engagement
9,view_search_results


In [11]:
def sql_executer(sql):
    data  = read_gbq(sql, project_id=project_id, credentials=credentials)
    return(data)

In [12]:
sql_executer(sql)

Downloading: 100%|██████████|


,event_name
0,view_item
1,page_view
2,scroll
3,select_promotion
4,purchase
5,view_item_list
6,begin_checkout
7,first_visit
8,view_promotion
9,select_item


In [13]:
event_names_list = event_names['event_name'].tolist()
event_names_info = ", ".join(event_names_list)
event_names_info

'page_view, scroll, select_promotion, purchase, view_promotion, select_item, view_item, session_start, user_engagement, view_search_results, add_payment_info, add_shipping_info, click, add_to_cart, first_visit, begin_checkout, view_item_list'

## Create Sql query from user question then return the resuls from DB


In [14]:

output_parser = StrOutputParser()

model = ChatOpenAI(openai_api_key = openai_api_key, model = 'gpt-3.5-turbo')

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are the best Data Analyst who is an expert in SQL and you work with GCP BigQuery all the time. You are working with e-commerce data and you provide the best analysis. You answer the user's question using the following table schema:\n\n{schema}\n\nand the possible event names are:\n\n{event_names}\n\nThe table is `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`. Please provide only the SQL query without any additional explanation, and do not wrap it in triple quotes."),
    ("user", "{question}")
])
chain = prompt_template | model | output_parser
chain_input = {
    "question": "Which Day we get most money",
    "schema": schema,
    "event_names": event_names_info
}
sql_query = chain.invoke(chain_input)
print(sql_query)

SELECT 
    event_date,
    SUM(ecommerce.purchase_revenue_in_usd) AS total_revenue
FROM 
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
WHERE 
    event_name = 'purchase'
GROUP BY 
    event_date
ORDER BY 
    total_revenue DESC
LIMIT 1;


In [15]:
sql_query

"SELECT \n    event_date,\n    SUM(ecommerce.purchase_revenue_in_usd) AS total_revenue\nFROM \n    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`\nWHERE \n    event_name = 'purchase'\nGROUP BY \n    event_date\nORDER BY \n    total_revenue DESC\nLIMIT 1;"

In [16]:
parsed_sql_query = output_parser.parse(sql_query)
clean_sql_query = parsed_sql_query.strip('/').strip("'").strip('"""').strip("```sql").strip("```").strip(' ```"').strip()
clean_sql_query = " ".join([line.strip() for line in clean_sql_query.splitlines()])
print(clean_sql_query)

SELECT event_date, SUM(ecommerce.purchase_revenue_in_usd) AS total_revenue FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` WHERE event_name = 'purchase' GROUP BY event_date ORDER BY total_revenue DESC LIMIT 1;


In [17]:
clean_sql_query

"SELECT event_date, SUM(ecommerce.purchase_revenue_in_usd) AS total_revenue FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*` WHERE event_name = 'purchase' GROUP BY event_date ORDER BY total_revenue DESC LIMIT 1;"

In [18]:
result = read_gbq(clean_sql_query, project_id=project_id, credentials=credentials)
result

Downloading: 100%|██████████|


,event_date,total_revenue
0,20201130,11990.0


## Tool - > DuckDuckSearch

In [19]:
##  Make search with DuckDuckSearch

from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from duckduckgo_search.duckduckgo_search import RatelimitException

wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", time="d" , max_results=10)

search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="news" )


try:
    # Sorguyu çalıştırın
    search = wrapper.results("Obama", max_results=10)  # Doğrudan wrapper üzerinden sonuçları çekin
    print(search)  # Sonuçları yazdırın
except RatelimitException:
    print("Ratelimit reached. Please wait before trying again.")


[{'snippet': 'Obama is the latest high-profile Democrat to visit Arizona, as the party puts on a full-court press with fewer than 20 days left in the race. Harris has stopped by twice in the past month, vice ...', 'title': "Obama uses withering mockery in Arizona as he questions Trump's ...", 'link': 'https://www.washingtonpost.com/politics/2024/10/18/obama-trump-arizona-rally-harris/'}, {'snippet': "TUCSON, Ariz. — Former President Barack Obama on Friday contrasted former President Donald Trump's character to that of the late Sen. John McCain during a packed rally at the University of ...", 'title': 'Barack Obama contrasts John McCain with Trump at Arizona rally - NBC News', 'link': 'https://www.nbcnews.com/politics/2024-election/barack-obama-john-mccain-trump-arizona-rally-rcna176205'}, {'snippet': "Barack Obama questioned Donald Trump's stamina for high office, as he campaigned on behalf of Democratic presidential candidate Kamala Harris in the swing state of Arizona.. At a Friday r

## Deep Dive Into Agent

In [20]:
##Define Custom Tools
search_tool = DuckDuckGoSearchRun() 

os.environ["OPENAI_API_KEY"] = openai_api_key


class BigQueryInput(BaseModel):
    query: str = Field(..., description="The SQL query to execute on BigQuery.")

def sql_executer(query: str):
    try:
        # SQL sorgusunu çalıştır
        data = read_gbq(query, project_id=project_id, credentials=credentials)
        return data.to_json()  # JSON olarak döndür
    except Exception as e:
        return f"Error occurred: {str(e)}"

bigquery_tool = Tool(
    name="BigQuery",
    func=sql_executer,
    description=(
        "Executes SQL queries on BigQuery and returns the results. "
        "Use this tool to run SQL queries on the provided database. "
        "Input should be a valid SQL query string."
    ),
    args_schema=BigQueryInput
)


# Agent 1: SQL Developer
sql_writer = Agent(
    role='SQL Developer/Data Analyst',
    goal="Provide insightful analysis to support the marketing/product teams.",
    backstory=(
        "You are an expert data analyst. Your job is to generate SQL queries based on database schema and analyze them "
        "to provide insights about user behavior. Use the BigQuery tool to execute your SQL queries and retrieve data."
        "Prepare a report with numbers, how perform web site last week vs last 2 week before that. Asuume today is 15.06.2020.We need analaysis with number"
    ),
    verbose=True,
    memory=True,
    tools=[bigquery_tool,search_tool]
)

task_sql_writer = Task(
    description=(
        "First, search the 'bigquery-public-data.ga4_obfuscated_sample_ecommerce' dataset to use search_tool, get knowlage about how to query this dataset."
        "Second, analyze the 'bigquery-public-data.ga4_obfuscated_sample_ecommerce' dataset to understand the schema. "
        "Once you have an understanding of the available tables and columns, generate SQL queries to extract insights "
        "based on user behavior events. Focus on understanding user interactions like product views, add-to-carts,installing user,earn money, selling count "
        "and purchases. Provide a detailed report with numbers and your findings. Asuume today is 15.06.2020"
    ),
    expected_output="A detailed report based on data analysis of user behavior and ecommerce events with numbers and compare last week metrics to last 2 week metric if there is increase or decrease is appear you can say the ratio.Your report about the numbers such as 'your revenue increased %20 for last 15 days'",
    agent=sql_writer
)




crew = Crew(
    agents=[sql_writer],
    tasks=[task_sql_writer],
    process=Process.sequential  
)

inputs = {
    "database": "bigquery-public-data.ga4_obfuscated_sample_ecommerce"
}
result = crew.kickoff(inputs=inputs)

# Sonuçları yazdır
print(result)

# Agent: SQL Developer/Data Analyst
## Task: First, search the 'bigquery-public-data.ga4_obfuscated_sample_ecommerce' dataset to use search_tool, get knowlage about how to query this dataset.Second, analyze the 'bigquery-public-data.ga4_obfuscated_sample_ecommerce' dataset to understand the schema. Once you have an understanding of the available tables and columns, generate SQL queries to extract insights based on user behavior events. Focus on understanding user interactions like product views, add-to-carts,installing user,earn money, selling count and purchases. Provide a detailed report with numbers and your findings. Asuume today is 15.06.2020
 

I encountered an error while trying to use the tool. This was the error: 1 validation error for DDGInput
query
  Field required [type=missing, input_value={'name': 'bigquery-public...cated_sample_ecommerce'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing.
 Tool duckduckgo_search accepts these 

In [21]:
search_tool = DuckDuckGoSearchRun()  
os.environ["OPENAI_API_KEY"] = openai_api_key


# Agent 1: Marketing Specialist
marketing_specialist = Agent(
    role='Marketing Specialist',
    goal="Devise optimal marketing strategies by analyzing user behavior and advertising performance.",
    backstory="you can make research use the search_tool about the marketing treds",
    verbose=True,
    memory=True,  # Önceki pazarlama stratejilerini hatırlar
    tools=[search_tool],
    allow_delegation=True
    
)


writer = Agent(
    role='News Writer',
    goal="Get info from marketing_specialist agent then write a news about new trends",
    backstory="You are the writer about marketing news, you have worked with best marketing company at he before",
    verbose=True,
    memory=True,  
     
)


task_marketing = Task(
    description="Search the most popular marketing trends used by popular companies.Search Appsflyer, adjust, sensortower, google trends etc",
    expected_output="A report about marketing trend",
    agent=marketing_specialist
)


task_writer = Task(
    description="Convert the report you received from your marketing specialist into plain text",
    expected_output=" A plain text which is going to publish on the marketing magazine",
    agent=writer
)


crew = Crew(
    agents=[marketing_specialist, writer],
    tasks=[task_marketing, task_writer],
    process=Process.sequential,  # Görevler sırayla çalışacak
    verbose = True
)


result = crew.kickoff()

# Sonuçları yazdır
print(result)

2024-10-19 13:54:03,223 - 9032 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Marketing Specialist
## Task: Search the most popular marketing trends used by popular companies.Search Appsflyer, adjust, sensortower, google trends etc


# Agent: Marketing Specialist
## Thought: I need to gather information about current popular marketing trends used by companies, utilizing tools like Appsflyer, Adjust, Sensortower, and Google Trends, among others.
## Using tool: duckduckgo_search
## Tool Input: 
"{\"query\": \"current popular marketing trends 2023 site:appsflyer.com OR site:adjust.com OR site:sensortower.com OR site:trends.google.com\"}"
## Tool Output: 
Their use constitutes nominative fair use under applicable trademark laws and does not imply any affiliation or endorsement by the respective owners. See the Top iOS Apps in US on the App Store Charts by Free, Paid, and Top Grossing. Sensor Tower provides a detailed visualization of the top ranking apps by category and country, along with app ... Last month's estimates were 10m downloads and $11m revenue. 

## Chat with manuel history added


In [22]:

from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage,SystemMessage

model = ChatOpenAI(openai_api_key = openai_api_key , model = 'gpt-3.5-turbo')
chat_history = []

system_message = SystemMessage(content= "You are the best historian in the world and your biggest pleasure is talking with history and answer historic question.") 
chat_history.append(system_message)

while True:
    query = input('You : ')
    if query.lower() == "exit":
        break
    chat_history.append(HumanMessage(content=query))    ## Add User Message to list chat_history
    result = model.invoke(chat_history)                 ## Generate AI message
    response = result.content                           ## Get Just Content from AI Message
    chat_history.append(AIMessage(content = response))  ## Add AI message to list chat_history
    print(f"human : {query}")                           ## Display Human Message
    print(f"AI    : {response}")                        ## Display AI Message


human : how do you explain the collapse the Ottomon Empire?
AI    : The collapse of the Ottoman Empire was a complex process that spanned several decades and can be attributed to a combination of internal and external factors. 

Internally, the empire faced challenges such as political corruption, economic stagnation, and social unrest. The system of government became increasingly outdated and inefficient, leading to a decline in the empire's ability to govern effectively. Additionally, the empire's diverse population, consisting of various ethnic and religious groups, posed challenges for maintaining unity and stability.

Externally, the Ottoman Empire faced increasing pressure from European powers, who sought to expand their influence in the region. The empire's military strength began to wane, making it vulnerable to external threats. The empire also struggled to modernize and adapt to the changing global landscape, falling behind technologically and economically compared to its Eur

In [23]:
model = ChatOpenAI(openai_api_key = openai_api_key , model = 'gpt-3.5-turbo')
from langchain.schema.runnable import RunnableLambda, RunnableSequence

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells koles about {topic}."),
        ("human", "Tell me {joke_count} jokes.")
    ]
) 
chain = prompt_template | model
result = chain.invoke({"topic" : "lawyer","joke_count" : 3})
print("----------result-----------")
print(result)
print("")
print("------ -----content---------")

print(result.content)


----------result-----------
content="1. Why did the lawyer bring a ladder to court? He heard the case was going to be heard on a higher level!\n\n2. How many lawyer jokes are there? Only three. The rest are true stories.\n\n3. Why don't lawyers play hide and seek? Good luck finding them when they're always behind the legal briefs!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 28, 'total_tokens': 97, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-2f760c07-5ac3-45c3-9174-bdbde81ed27b-0' usage_metadata={'input_tokens': 28, 'output_tokens': 69, 'total_tokens': 97, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}

------ -----content---------
1. Why did the lawyer bring a ladder 

In [24]:
## If you don't want to use LCEL you can use Runnable sequence

model = ChatOpenAI(openai_api_key = openai_api_key , model = 'gpt-3.5-turbo')
from langchain.schema.runnable import RunnableLambda, RunnableSequence

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a comedian who tells koles about {topic}."),
        ("human", "Tell me {joke_count} jokes.")
    ]
) 

format_promts = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model =RunnableLambda(lambda x : model.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

chain = RunnableSequence(first= format_promts,
                         middle=[invoke_model],
                         last= parse_output)

result = chain.invoke({"topic" : "lawyers","joke_count" : 3})
print("----------result-----------")
print(result)
print("")


----------result-----------
1. Why did the lawyer bring a ladder to court? Because he heard the case was being held on the second floor!

2. How does a lawyer sleep? First, he lies on one side, then he lies on the other.

3. Why don't lawyers play hide and seek? Because good luck finding one that actually wants to hide!



## RunnableParallel

### Basics

In [25]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI(openai_api_key = openai_api_key, model = "gpt-3.5-turbo")

joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain) 
result = map_chain.invoke({"topic": "bear"})
print("Joke:", result["joke"].content)
print("Poem:", result["poem"].content)


Joke: Why don't bears like fast food? Because they can't catch it!
Poem: Majestic and wild, the bear roams free,
In the forest's embrace, where it longs to be.


### More Specisific Example

In [26]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

model = ChatOpenAI(openai_api_key = openai_api_key, model = "gpt-3.5-turbo")

prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You are an expert product rewiever"),
     ("human", "List the main features of the product {product_name}."),
    ]
    
)

## Making prompt template output for pros
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            ("human","Given these features : {features}, list the pros of these features.")
        ]
    )
    return pros_template.format_prompt(features = features)


def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            ("human","Given these features : {features}, list the cons of these features.")
        ]
    )
    return cons_template.format_prompt(features = features)


def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"


pros_branch = (
    RunnableLambda(lambda x : analyze_pros(x))| model | StrOutputParser()
)

cons_branch = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)
    

In [27]:
analyze_pros('example')

ChatPromptValue(messages=[SystemMessage(content='You are an expert product reviewer.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Given these features : example, list the pros of these features.', additional_kwargs={}, response_metadata={})])

In [28]:
analyze_cons('example')

ChatPromptValue(messages=[SystemMessage(content='You are an expert product reviewer.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Given these features : example, list the cons of these features.', additional_kwargs={}, response_metadata={})])

In [29]:
# Make Chain & Get the result 

chain = (
    prompt_template
    |model
    |StrOutputParser()
    |RunnableParallel(branches = {"pros" : pros_branch, "cons" : cons_branch})
    |RunnableLambda(lambda x : combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

result = chain.invoke({"product_name" : "Macbook Pro"})
print(result)

Pros:
Pros of the MacBook Pro features:

1. Retina display: Provides stunning visuals with high-resolution and vibrant colors for an immersive viewing experience.
2. Powerful performance: Intel Core processors ensure fast and efficient performance, ideal for multitasking and demanding tasks.
3. Touch Bar: Enhances user interaction by providing quick access to tools and functions, improving workflow efficiency.
4. Touch ID: Offers secure and convenient authentication, adding an extra layer of security to the device.
5. Thunderbolt 3 ports: High-speed data transfer and versatile connectivity options for seamless integration with various peripherals.
6. macOS operating system: Stable, user-friendly interface with access to a wide range of applications and software for productivity and creativity.
7. SSD storage: Provides fast read/write speeds, quick boot times, and reliable storage performance for enhanced overall system responsiveness.
8. Thin and lightweight design: Portable and sleek 

## RunableBranch
Generate Message according to user review

In [30]:
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain.schema.runnable import RunnableBranch


model = ChatOpenAI(openai_api_key = openai_api_key, model = "gpt-3.5-turbo")

positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpfull AI asistant."),
        ("human", "Generate a thank you note for this positive feedback {feedbavk}")
    ]
)

negaitve_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpfull AI asistant"),
        ("human", "Generate response addressing this negative feedback : {feedback}")
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)



classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate: {feedback}."),
    ]
)


branches = RunnableBranch(
    (
        lambda x : "positive" in x,
        positive_feedback_template | model | StrOutputParser()
    ),
    (
        lambda x : "negative" in x,
        negaitve_feedback_template | model | StrOutputParser()
    ),
    (
        lambda x :"neutral" in x ,
        neutral_feedback_template| model | StrOutputParser()
    ),
    escalate_feedback_template | model | StrOutputParser() 
    
)

classification_chain = classification_template | model | StrOutputParser()

chain = classification_chain | branches

review =  input("What is your review about product:")

result = chain.invoke({"feedback" : review})
print(result)

I'm sorry to hear that you found the feedback negative. Your opinion is valuable, and I appreciate your honesty. If there are specific areas you believe could be improved upon, please let me know so that we can work on addressing them. Thank you for sharing your thoughts.
